In [ ]:
import pandas as pd
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import colors
import glob
import seaborn as sns
max_diffuse_nsb_std = 2.3

start_date_2019 = datetime.strptime("2019-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2019 = datetime.strptime("2019-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
    
start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

colors = [
	'#8fbbda', # Pastel Blue
	'#ffbf86', # Pastel Orange
	'#96d096', # Pastel Green
	'#eb9393', # Pastel Red
	'#cab3de', # Pastel Purple
	'#f1bbe0', # Pastel Pink
	'#c6aba5', # Pastel Brown 
]

In [ ]:
datacheck_files = glob.glob('/Users/vdk/muons2024/v0.9-v0.10/20*/DL1_datacheck_*.h5')
datacheck_files.sort()

runsummary = []
cosmics = []
cis = []
for file in datacheck_files:
    try:
        runsummary.append(pd.read_hdf(file, 'runsummary'))
        cosmics.append(pd.read_hdf(file, 'cosmics'))
        cis.append(pd.read_hdf(file, 'cosmics_intensity_spectrum'))
    except:
        print(file)
    
cosmics_pd = pd.concat(cosmics, ignore_index=True)
runsummary_pd = pd.concat(runsummary, ignore_index=True)
cis_pd = pd.concat(cis, ignore_index=True)
cosmics_pd.columns

#### Only low NSB runs

In [ ]:
low_nsb_runs = cis_pd[['yyyymmdd','runnumber', 'subrun']][(cis_pd['diffuse_nsb_std'] <= max_diffuse_nsb_std)]
low_nsb_runsummary = runsummary_pd[runsummary_pd['runnumber'].isin(low_nsb_runs['runnumber'])]
low_nsb_cosmics = cosmics_pd[cosmics_pd['runnumber'].isin(low_nsb_runs['runnumber'])]
low_nsb_cosmics

#### Only high NSB runs

In [ ]:
high_nsb_runs = cis_pd[['yyyymmdd','runnumber', 'subrun']][(cis_pd['diffuse_nsb_std'] > max_diffuse_nsb_std)]
high_nsb_runsummary = runsummary_pd[runsummary_pd['runnumber'].isin(high_nsb_runs['runnumber'])]
high_nsb_cosmics = cosmics_pd[cosmics_pd['runnumber'].isin(high_nsb_runs['runnumber'])]
high_nsb_runsummary

#### Evolution of parameters

In [ ]:
# Define the years and their corresponding start and end dates
years = {
    2019: (start_date_2019, end_date_2019),
    2020: (start_date_2020, end_date_2020),
    2021: (start_date_2021, end_date_2021),
    2022: (start_date_2022, end_date_2022),
    2023: (start_date_2023, end_date_2023),
    2024: (start_date_2024, end_date_2024)
}

# Initialize dictionaries to store results
mu_eff_mean = {}
mu_eff_std = {}
mu_intensity_mean = {}
mu_width_mean = {}
mu_width_std = {}
mu_radius_mean = {}
mu_radius_std = {}
runs = {}

# Iterate over the years and calculate the required statistics
for year, (start_date, end_date) in years.items():
    mask = (low_nsb_runsummary['time'] > start_date) & (low_nsb_runsummary['time'] < end_date)
    mu_eff_mean[year] = low_nsb_runsummary['mu_effi_mean'][mask]
    mu_eff_std[year] = low_nsb_runsummary['mu_effi_stddev'][mask]
    mu_intensity_mean[year] = low_nsb_runsummary['mu_intensity_mean'][mask]
    mu_width_mean[year] = low_nsb_runsummary['mu_width_mean'][mask]
    mu_width_std[year] = low_nsb_runsummary['mu_width_stddev'][mask]
    runs[year] = low_nsb_runsummary['runnumber'][mask]

    mask_cosmics = (low_nsb_cosmics['time'] > start_date) & (low_nsb_cosmics['time'] < end_date)
    mu_radius_mean[year] = low_nsb_cosmics['mu_radius_mean'][mask_cosmics]
    mu_radius_std[year] = low_nsb_cosmics['mu_radius_stddev'][mask_cosmics]

# Print the results rounded to three decimal places
for year in years.keys():
    mean_eff = round(np.mean(mu_eff_mean[year]), 3)
    std_eff = round(np.mean(mu_eff_std[year]), 3)
    mean_intensity = round(np.mean(mu_intensity_mean[year]), 3)
    mean_width = round(np.mean(mu_width_mean[year]), 3)
    std_width = round(np.mean(mu_width_std[year]), 3)
    mean_radius = round(np.mean(mu_radius_mean[year]), 3)
    std_radius = round(np.mean(mu_radius_std[year]), 3)
    
    print(f"===============YEAR {year}==========================\n")
    print(f"Mean opt eff for {year} year = {mean_eff} with std = {std_eff}")
    print(f"Mean ring size for {year} year = {mean_intensity}")
    print(f"Mean width for {year} year = {mean_width} with std = {std_width}")
    print(f"Mean radius for {year} year = {mean_radius} & {std_radius}")
    print("\n==================================================\n")

In [ ]:
# Initialize the plot
colors_background = ['blue', 'green', 'orange', 'purple', 'yellow', 'cyan']
ytext = 0.08
plt.figure(figsize=(12, 9))
#sns.regplot(x=low_nsb_runsummary['runnumber'], y=low_nsb_runsummary['mu_effi_mean'], color='k', scatter_kws={'s': 10}, label = 'Optical efficiency value per run (Low NSB data)')
sns.regplot(x=high_nsb_runsummary['runnumber'], y=high_nsb_runsummary['mu_effi_mean'], color='g', scatter_kws={'s': 10}, label = 'Optical efficiency value per run (High NSB data)')
#sns.regplot(x=runsummary_pd['runnumber'], y=runsummary_pd['mu_effi_mean'], color='g', scatter_kws={'s': 10})

# Use a loop to fill between the run number ranges for each year
x1 = 0
for idx, (year, runs_year) in enumerate(runs.items()):
    x2 = max(runs_year)
    plt.fill_betweenx(y=[0, 0.3], x1=x1, x2=x2, color=colors_background[idx], alpha=0.045)
    plt.annotate(str(year), (((x1 + x2) / 2)-150, ytext), c='red')
    x1 = x2

plt.fill_betweenx(y=[0, 0.3], x1=max(runs[2024]), x2=20000, color=colors_background[idx], alpha=0.045)
plt.grid(alpha=0.2)
plt.ylim(0.075, 0.3)
plt.xlim(0, 17500)
plt.legend()
plt.xlabel('Runnumber')
plt.ylabel('Mean optical efficiency')
#plt.savefig('/Users/vdk/Software/code/muon_paper_2024/figures/OpticalEfficiencyEvolutionHighNSB.pdf', dpi=200, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# Initialize the plot
colors_background = ['blue', 'green', 'orange', 'purple', 'yellow', 'cyan']
ytext = 100
plt.figure(figsize=(12, 9))
#sns.regplot(x=low_nsb_runsummary['runnumber'], y=low_nsb_runsummary['mu_intensity_mean'], color='k', scatter_kws={'s': 10}, label = 'Mean ring size per run (Low NSB data)')
sns.regplot(x=high_nsb_runsummary['runnumber'], y=high_nsb_runsummary['mu_intensity_mean'], color='g', scatter_kws={'s': 10}, label = 'Mean ring size per run (High NSB data)')
#sns.regplot(x=runsummary_pd['runnumber'], y=runsummary_pd['mu_effi_mean'], color='g', scatter_kws={'s': 10})

# Use a loop to fill between the run number ranges for each year
x1 = 0
for idx, (year, runs_year) in enumerate(runs.items()):
    x2 = max(runs_year)
    plt.fill_betweenx(y=[0, 4000], x1=x1, x2=x2, color=colors_background[idx], alpha=0.045)
    plt.annotate(str(year), (((x1 + x2) / 2)-150, ytext), c='red')
    x1 = x2

plt.fill_betweenx(y=[0,3500], x1=max(runs[2024]), x2=20000, color=colors_background[idx], alpha=0.045)
plt.grid(alpha=0.2)
plt.ylim(0, 3500)
plt.xlim(0, 17500)
plt.legend(loc='upper left')
plt.xlabel('Runnumber')
plt.ylabel('Mean ring size [p.e.]')
#plt.savefig('/Users/vdk/Software/code/muon_paper_2024/figures/RingSizeEvolutionHighNSB.pdf', dpi=200, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# Initialize the plot
colors_background = ['blue', 'green', 'orange', 'purple', 'yellow', 'cyan']
ytext = 0.01
upper_bound = 0.3
lower_bound = 0
plt.figure(figsize=(12, 9))
sns.regplot(x=low_nsb_runsummary['runnumber'], y=low_nsb_runsummary['mu_width_mean'], color='k', scatter_kws={'s': 10}, label = 'Mean ring width per run (Low NSB data)',)# x_bins = 400)
#sns.regplot(x=high_nsb_runsummary['runnumber'], y=high_nsb_runsummary['mu_width_mean'], color='g', scatter_kws={'s': 10}, label = 'Mean ring width per run (High NSB data)')#, x_bins = 400)
#sns.regplot(x=runsummary_pd['runnumber'], y=runsummary_pd['mu_effi_mean'], color='g', scatter_kws={'s': 10})

# Use a loop to fill between the run number ranges for each year
x1 = 0
for idx, (year, runs_year) in enumerate(runs.items()):
    x2 = max(runs_year)
    plt.fill_betweenx(y=[0, upper_bound], x1=x1, x2=x2, color=colors_background[idx], alpha=0.045)
    plt.annotate(str(year), (((x1 + x2) / 2)-150, ytext), c='red')
    x1 = x2

plt.fill_betweenx(y=[0,upper_bound], x1=max(runs[2024]), x2=20000, color=colors_background[idx], alpha=0.045)
plt.grid(alpha=0.2)
plt.ylim(0, upper_bound)
plt.xlim(0, 17500)
plt.legend(loc='upper left')
plt.xlabel('Runnumber')
plt.ylabel('Mean ring width [deg]')
#plt.savefig('/Users/vdk/Software/code/muon_paper_2024/figures/RingWidthEvolutionLowNSB.pdf', dpi=200, format='pdf', bbox_inches='tight')

plt.show()

In [ ]:
# Initialize the plot
colors_background = ['blue', 'green', 'orange', 'purple', 'yellow', 'cyan']
ytext = 0.8
upper_bound = 1.5
lower_bound = 0.75

grouped_subruns_low = low_nsb_cosmics.groupby(['runnumber', 'subrun'])['mu_radius_mean'].mean().reset_index()
grouped_subruns_high = high_nsb_cosmics.groupby(['runnumber', 'subrun'])['mu_radius_mean'].mean().reset_index()

# Step 2: Group by 'runnumber' again to get the mean radius over all subruns for each runnumber
mean_radius_per_runnumber_low = grouped_subruns_low.groupby('runnumber')['mu_radius_mean'].mean().reset_index()
mean_radius_per_runnumber_high = grouped_subruns_high.groupby('runnumber')['mu_radius_mean'].mean().reset_index()

plt.figure(figsize=(12, 9))
sns.regplot(x=mean_radius_per_runnumber_low['runnumber'], y=mean_radius_per_runnumber_low['mu_radius_mean'], color='k', scatter_kws={'s': 10}, label = 'Mean ring width per run (Low NSB data)',)# x_bins = 400)
#sns.regplot(x=mean_radius_per_runnumber_high['runnumber'], y=mean_radius_per_runnumber_high['mu_radius_mean'], color='g', scatter_kws={'s': 10}, label = 'Mean ring width per run (High NSB data)')#, x_bins = 400)
#sns.regplot(x=runsummary_pd['runnumber'], y=runsummary_pd['mu_effi_mean'], color='g', scatter_kws={'s': 10})

# Use a loop to fill between the run number ranges for each year
x1 = 0
for idx, (year, runs_year) in enumerate(runs.items()):
    x2 = max(runs_year)
    plt.fill_betweenx(y=[0, upper_bound], x1=x1, x2=x2, color=colors_background[idx], alpha=0.045)
    plt.annotate(str(year), (((x1 + x2) / 2)-150, ytext), c='red')
    x1 = x2

plt.fill_betweenx(y=[0,upper_bound], x1=max(runs[2024]), x2=20000, color=colors_background[idx], alpha=0.045)
plt.grid(alpha=0.2)
plt.ylim(lower_bound, upper_bound)
plt.xlim(0, 17500)
plt.legend(loc='upper left')
plt.xlabel('Runnumber')
plt.ylabel('Mean ring radius [deg]')
#plt.savefig('/Users/vdk/Software/code/muon_paper_2024/figures/RingRadiusEvolutionLowNSB.pdf', dpi=200, format='pdf', bbox_inches='tight')

plt.show()

#### MC to DATA

In [ ]:
size_outside = 500
ring_size_upper = 10000
ring_completeness_cut = 0.9
ring_containment_cut = 0.9
bin_number = 8

start_date_2019 = datetime.strptime("2019-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2019 = datetime.strptime("2019-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()


colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
]

def format_axes(ax,option):
    
    if(option=='width'):
        ax.set_xlabel('Muon radius [deg]')
        ax.set_ylabel('Muon ring width [deg]')
        ax.grid(linestyle="dashed")
        ax.set_ylim(0.042,0.14)
        ax.set_xlim(1.0,1.25)
        ax.legend()
    if(option=='size'):
        ax2.grid(linestyle="dashed")
        ax2.legend(ncol=2)
        ax2.set_xlabel('Muon radius [deg]')
        ax2.set_ylabel('Muon Size [pe]')
        ax2.set_ylim(2000,4000)
        ax2.set_xlim(1.0,1.25)

In [ ]:
muon_file = '/Users/vdk/muons2024/highNSB_fits_filtered.csv'
df_all_data = pd.read_csv(muon_file, na_values=['NA', '?'])
df_good_data_real_high_nsb = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['size_outside'] < 500)]
del(df_all_data)

In [ ]:


muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[:]] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data_real_low_nsb = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['size_outside'] < 500)]
del(df_all_data)
df_good_data_real_low_nsb

In [ ]:
listdir_z0_cta1 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/cta-config/fit_valid/tables/*")
listdir_z0_cta2 = glob.glob("/Users/vdk/muons2024/lapalma_simulations/cta-config/tables_pt2/*")

listdir_low_nsb = glob.glob("/Users/vdk/muons2024/lapalma_simulations/nsb_play/0.5/table/*")
listdir_high_nsb = glob.glob("/Users/vdk/muons2024/lapalma_simulations/nsb_play/x1.5NSB/*")
listdir_3xNSB = glob.glob("/Users/vdk/muons2024/lapalma_simulations/nsb_play/3xNSB/tables/*")

listdir_fitrandom_initial = glob.glob("/Users/vdk/muons2024/lapalma_simulations/fit_play/fit_random_initial_orig/tables/*")

listdir_perf_paper = glob.glob("/Users/vdk/muons2024/lapalma_simulations/performance_paper/tables/*")



listdir = listdir_z0_cta1+listdir_z0_cta2
dat = Table.read('{}'.format(listdir[0]), format='fits')
for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat2['is_valid'] = dat2['is_valid'].astype(bool)
    dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)
    dat = vstack([dat, dat2])
df_2 = dat.to_pandas()
df_good_data_sim = df_2[df_2['good_ring'] & (df_2['muon_efficiency'] < 1) & (df_2['size_outside'] < size_ouside)]


df_good_data_sim


In [ ]:
start_date_2019 = datetime.strptime("2019-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2019 = datetime.strptime("2019-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

ring_size_upper = 10000
ring_completeness_cut = 0.9
ring_containment_cut = 0.9
bin_number = 8


colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
]

def format_axes(ax,option):
    
    if(option=='width'):
        ax.set_xlabel('Muon radius [deg]')
        ax.set_ylabel('Muon ring width [deg]')
        ax.grid(linestyle="dashed")
        ax.set_ylim(0.042,0.14)
        ax.set_xlim(1.0,1.25)
        ax.legend()
    if(option=='size'):
        ax2.grid(linestyle="dashed")
        ax2.legend(ncol=2)
        ax2.set_xlabel('Muon radius [deg]')
        ax2.set_ylabel('Muon Size [pe]')
        ax2.set_ylim(2000,4000)
        ax2.set_xlim(1.0,1.25)

alpha_reg = 0.2

plt.figure(figsize = (7,7))

flag = '2023-24'

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
#                                                  (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2019) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2019)], 
#             y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
#                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2019) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2019)], 
#             x_bins=bin_number, color=colors[0], label = 'Data for 2019 year', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
#                                                  (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2020) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2020)], 
#             y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
#                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2020) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2020)], 
#             x_bins=bin_number, color=colors[1], label = 'Data for 2020 year', scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
#                                                  (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2021) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2021)], 
#             y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
#                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2021) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2021)], 
#             x_bins=bin_number, color=colors[2], label = 'Data for 2021 year', scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
#                                                  (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2022) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2022)], 
#             y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
#                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2022) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2022)], 
#             x_bins=bin_number, color=colors[3], label = 'Data for 2022 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
                                                (df_good_data_real_high_nsb['event_time'] > start_date_2023) &
                                                (df_good_data_real_high_nsb['event_time'] < end_date_2023)], 
            y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
                                                (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
                                                (df_good_data_real_high_nsb['event_time'] > start_date_2023) &
                                                (df_good_data_real_high_nsb['event_time'] < end_date_2023)], 
            x_bins=bin_number, color=colors[4], label = 'Data for 2023 year', scatter_kws={'alpha':1}, line_kws={'alpha':1})
sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)&
                                                 (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
                                                (df_good_data_real_high_nsb['event_time'] > start_date_2024) &
                                                (df_good_data_real_high_nsb['event_time'] < end_date_2024)], 
            y = df_good_data_real_high_nsb['ring_width'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper)& 
                                                (df_good_data_real_high_nsb['muon_efficiency'] < 1) &
                                                (df_good_data_real_high_nsb['event_time'] > start_date_2024) &
                                                (df_good_data_real_high_nsb['event_time'] < end_date_2024)], 
            x_bins=bin_number, color=colors[5], label = 'Data for 2024 year', scatter_kws={'alpha':1}, line_kws={'alpha':1})
sns.regplot(x = df_good_data_sim['ring_radius'][(df_good_data_sim['ring_completeness'] > ring_completeness_cut) & 
                                            (df_good_data_sim['ring_containment'] > ring_containment_cut) & 
                                            (df_good_data_sim['size_outside'] < size_outside) & 
                                            (df_good_data_sim['ring_size'] < ring_size_upper)  & 
                                            (df_good_data_sim['muon_efficiency'] < 1)], 
            y = df_good_data_sim['ring_width'][(df_good_data_sim['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data_sim['ring_containment'] > ring_containment_cut) &  
                                           (df_good_data_sim['size_outside'] < size_outside) & 
                                           (df_good_data_sim['ring_size'] < ring_size_upper)  & 
                                           (df_good_data_sim['muon_efficiency'] < 1)], 
            x_bins=bin_number, color='k', label = 'Simulations with nominal NSB (0.24 GHz)')





# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2019) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2019)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2019) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2019)], 
#             x_bins=bin_number, color=colors[0], label = 'Data for 2019 year', scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2020) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2020)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2020) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2020)], 
#             x_bins=bin_number, color=colors[1], label = 'Data for 2020 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2021) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2021)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2021) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2021)], 
#             x_bins=bin_number, color=colors[2], label = 'Data for 2021 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2022) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2022)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2022) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2022)], 
#             x_bins=bin_number, color=colors[3], label = 'Data for 2022 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})
# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2023) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2023)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2023) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2023)], 
#             x_bins=bin_number, color=colors[4], label = 'Data for 2023 year', scatter_kws={'alpha':1}, line_kws={'alpha':1})
# sns.regplot(x = df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
#                                                  (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
#                                                  (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2024) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2024)], 
#             y = df_good_data_real_high_nsb['ring_size'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) &  
#                                                (df_good_data_real_high_nsb['size_outside'] < size_outside) & 
#                                                (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) & 
#                                                 (df_good_data_real_high_nsb['event_time'] > start_date_2024) &
#                                                 (df_good_data_real_high_nsb['event_time'] < end_date_2024)], 
#             x_bins=bin_number, color=colors[5], label = 'Data for 2024 year')

# # # sns.regplot(x = df_good_data_high_nsb['ring_radius'][(df_good_data_high_nsb['ring_completeness'] > 0.9) &
# # #                                             (df_good_data_high_nsb['ring_containment'] > ring_containment_cut) &  
# # #                                             (df_good_data_high_nsb['size_outside'] < size_outside) & 
# # #                                             (df_good_data_high_nsb['ring_size'] < ring_size_upper) & 
# # #                                             (df_good_data_high_nsb['muon_efficiency'] < 1)], 
# # #             y = df_good_data_high_nsb['ring_size'][(df_good_data_high_nsb['ring_completeness'] > 0.9) & 
# # #                                           (df_good_data_high_nsb['ring_containment'] > ring_containment_cut) & 
# # #                                           (df_good_data_high_nsb['size_outside'] < size_outside) & 
# # #                                           (df_good_data_high_nsb['ring_size'] < ring_size_upper) & 
# # #                                           (df_good_data_high_nsb['muon_efficiency'] < 1)], 
# # #             x_bins=bin_number, color='r', label = 'High NSb data')

sns.regplot(x = df_good_data_sim['ring_radius'][(df_good_data_sim['ring_completeness'] > 0.9) &
                                            (df_good_data_sim['ring_containment'] > ring_containment_cut) &  
                                            (df_good_data_sim['size_outside'] < size_outside) & 
                                            (df_good_data_sim['ring_size'] < ring_size_upper) & 
                                            (df_good_data_sim['muon_efficiency'] < 1)], 
            y = df_good_data_sim['ring_size'][(df_good_data_sim['ring_completeness'] > 0.9) & 
                                          (df_good_data_sim['ring_containment'] > ring_containment_cut) & 
                                          (df_good_data_sim['size_outside'] < size_outside) & 
                                          (df_good_data_sim['ring_size'] < ring_size_upper) & 
                                          (df_good_data_sim['muon_efficiency'] < 1)], 
            x_bins=bin_number, color='k', label = 'Simulations with nominal NSB (0.24 GHz)')

# sns.regplot(x = df_good_data_2['ring_radius'][(df_good_data_2['ring_completeness'] > 0.9) &
#                                             (df_good_data_2['ring_containment'] > ring_containment_cut) &  
#                                             (df_good_data_2['size_outside'] < size_outside) & 
#                                             (df_good_data_2['ring_size'] < ring_size_upper) & 
#                                             (df_good_data_2['muon_efficiency'] < 1)], 
#             y = df_good_data_2['ring_size'][(df_good_data_2['ring_completeness'] > 0.9) & 
#                                           (df_good_data_2['ring_containment'] > ring_containment_cut) & 
#                                           (df_good_data_2['size_outside'] < size_outside) & 
#                                           (df_good_data_2['ring_size'] < ring_size_upper) & 
#                                           (df_good_data_2['muon_efficiency'] < 1)], 
#             x_bins=bin_number, color='cyan', label = '1xNSB')


plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon Size [pe]')
plt.grid(linestyle="dashed")
plt.ylim(2000,3500)
plt.xlim(0.9,1.25)

# plt.xlabel('Muon radius [deg]')
# plt.ylabel('Muon ring width [deg]')
# plt.grid(linestyle="dashed")
# plt.ylim(0.0,0.15)
# plt.xlim(0.9,1.25)

plt.legend(prop={'size': 12}, loc=2)
#plt.show()
#plt.savefig(f'/Users/vdk/Software/code/muon_paper_2024/figures/high_nsb_data_size_{flag}_NSBx1.png', dpi=200)
#plt.savefig(f'/Users/vdk/Software/code/muon_paper_2024/figures/high_nsb_data_width_{flag}_NSBx1.png', dpi=200)

In [ ]:
df_good_data_real_high_nsb['ring_radius'][(df_good_data_real_high_nsb['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real_high_nsb['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real_high_nsb['size_outside'] < size_outside) &  
                                                 (df_good_data_real_high_nsb['ring_size'] < ring_size_upper) &
                                                (df_good_data_real_high_nsb['event_time'] > start_date_2024) &
                                                (df_good_data_real_high_nsb['event_time'] < end_date_2024)]